In [1]:
# down postion station
# https://drive.google.com/file/d/1S_-0yAFAYiqDcHwUnUbwpTapU7UDCgje/view?usp=drive_link
!gdown 1S_-0yAFAYiqDcHwUnUbwpTapU7UDCgje

Downloading...
From: https://drive.google.com/uc?id=1S_-0yAFAYiqDcHwUnUbwpTapU7UDCgje
To: /content/S_pos_station.csv
100% 151k/151k [00:00<00:00, 69.7MB/s]


In [2]:
!pip install meteostat

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
from meteostat import Stations, Hourly
from meteostat.units import fahrenheit, direction, condition
from meteostat import Monthly
from meteostat import Point, Daily
import matplotlib.pyplot as plt

In [4]:
df_position_station = pd.read_csv("/content/S_pos_station.csv")
# df_position_station['time'] = pd.to_datetime(df_position_station['date_str'])
print(df_position_station.shape)
df_position_station.head()

(1635, 10)


,Country,Year,Sampling Point Id,Air Quality Station EoI Code,Longitude,Latitude,Altitude,Air Quality Station Area,Air Quality Station Type,AirQualityStation
0,Albania,2023.0,SPO-AL0203A_00001_100,AL0203A,20.78018,40.62593,848.0,suburban,background,STA-AL0203A
1,Albania,2023.0,SPO-AL0205A_00001_100,AL0205A,19.44920,41.31990,1.0,urban,traffic,STA-AL0205A
2,Albania,2023.0,SPO-AL0206A_00001_100,AL0206A,19.52342,42.31390,13.0,urban,background,STA-AL0206A
3,Albania,2023.0,SPO-AL0207A_00001_100,AL0207A,20.08170,41.11098,150.0,urban,traffic,STA-AL0207A
4,Bosnia and Herzegovina,2023.0,SPO-BA0029A_00001_100,BA0029A,18.42306,43.86750,630.0,urban,background,STA-BA0029A


In [ ]:
df_weather = df_position_station[['AirQualityStation', 'date_str', 'Latitude', 'Longitude']]
df_weather['time'] = pd.to_datetime(df_weather['date_str'])
for col in ['tavg', 'prcp', 'wspd', 'pres', 'tsun']:
    if col not in df_weather.columns:
        df_weather[col] = np.nan

df_weather.head()

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

,AirQualityStation,date_str,Latitude,Longitude,time,tavg,prcp,wspd,pres,tsun
0,STA-XK0009A,10-2018,42.4678,21.46043,2018-10-01,NaN,NaN,NaN,NaN,NaN
1,STA-XK0009A,11-2018,42.4678,21.46043,2018-11-01,NaN,NaN,NaN,NaN,NaN
2,STA-XK0009A,12-2018,42.4678,21.46043,2018-12-01,NaN,NaN,NaN,NaN,NaN
3,STA-XK0009A,01-2019,42.4678,21.46043,2019-01-01,NaN,NaN,NaN,NaN,NaN
4,STA-XK0009A,02-2019,42.4678,21.46043,2019-02-01,NaN,NaN,NaN,NaN,NaN


In [ ]:
for i in range(len(df_weather)):
    # Set time period
    start = df_weather['time'].iloc[i]

    position = (df_weather['Latitude'].iloc[i], df_weather['Longitude'].iloc[i])

    # Get Hourly data
    stations = Stations()
    stations = stations.nearby(position[0], position[1])
    station = stations.fetch(1)

    station_id = station.index[0]
    date_tuple = (start.year, start.month, start.day)

    start = datetime(date_tuple[0], date_tuple[1], date_tuple[2])
    end = start

    # Get Monthly data
    data = Monthly(station_id, start, end)
    data = data.fetch()

        # Kiểm tra nếu data không rỗng
    if not data.empty:
        df_weather.at[i, 'tavg'] = data['tavg'].iloc[0]
        df_weather.at[i, 'prcp'] = data['prcp'].iloc[0]
        df_weather.at[i, 'wspd'] = data['wspd'].iloc[0]
        df_weather.at[i, 'pres'] = data['pres'].iloc[0]
        df_weather.at[i, 'tsun'] = data['tsun'].iloc[0]
    else:
        # Nếu không có dữ liệu, gán giá trị NaN (hoặc giá trị mặc định khác)
        df_weather.at[i, 'tavg'] = np.nan
        df_weather.at[i, 'prcp'] = np.nan
        df_weather.at[i, 'wspd'] = np.nan
        df_weather.at[i, 'pres'] = np.nan
        df_weather.at[i, 'tsun'] = np.nan

In [ ]:
print(df_weather.shape)
df_weather.head()

(1958, 10)


,AirQualityStation,date_str,Latitude,Longitude,time,tavg,prcp,wspd,pres,tsun
0,STA-XK0009A,10-2018,42.4678,21.46043,2018-10-01,NaN,NaN,NaN,NaN,NaN
1,STA-XK0009A,11-2018,42.4678,21.46043,2018-11-01,NaN,NaN,NaN,NaN,NaN
2,STA-XK0009A,12-2018,42.4678,21.46043,2018-12-01,NaN,NaN,NaN,NaN,NaN
3,STA-XK0009A,01-2019,42.4678,21.46043,2019-01-01,NaN,NaN,NaN,NaN,NaN
4,STA-XK0009A,02-2019,42.4678,21.46043,2019-02-01,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_weather.to_csv("weather.csv", index=False)